In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_community.callbacks.manager import get_openai_callback
from langchain.agents import AgentExecutor, create_tool_calling_agent, load_tools, initialize_agent,AgentType

c:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash").bind(logprobs=True)
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo").bind(logprobs=True)

<h3>Get Log Probabilities</h3>

In [4]:
# Does not work for Gemini

In [5]:
msg=llmGemini.invoke(input=("human","how are you today?"))

TypeError: GenerativeServiceClient.generate_content() got an unexpected keyword argument 'logprobs'

In [6]:
# Works for OpenAI Models
msg=llmOpenAI.invoke(input=("human","how are you today?"))

In [7]:
msg.response_metadata

{'token_usage': {'completion_tokens': 23,
  'prompt_tokens': 17,
  'total_tokens': 40,
  'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}},
 'model_name': 'gpt-3.5-turbo-0125',
 'system_fingerprint': None,
 'finish_reason': 'stop',
 'logprobs': {'content': [{'token': 'I',
    'bytes': [73],
    'logprob': -0.2137053,
    'top_logprobs': []},
   {'token': "'m",
    'bytes': [39, 109],
    'logprob': -0.33866894,
    'top_logprobs': []},
   {'token': ' just',
    'bytes': [32, 106, 117, 115, 116],
    'logprob': -0.24022645,
    'top_logprobs': []},
   {'token': ' a',
    'bytes': [32, 97],
    'logprob': -0.0014491602,
    'top_logprobs': []},
   {'token': ' computer',
    'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
    'logprob': -0.05092288,
    'top_logprobs': []},
   {'token': ' program',
    'bytes': [32, 112, 114, 111, 103, 114, 97, 109],
    'logprob': -4.7875077e-05,
    't

In [8]:
msg.response_metadata["logprobs"]['content']

[{'token': 'I', 'bytes': [73], 'logprob': -0.2137053, 'top_logprobs': []},
 {'token': "'m",
  'bytes': [39, 109],
  'logprob': -0.33866894,
  'top_logprobs': []},
 {'token': ' just',
  'bytes': [32, 106, 117, 115, 116],
  'logprob': -0.24022645,
  'top_logprobs': []},
 {'token': ' a',
  'bytes': [32, 97],
  'logprob': -0.0014491602,
  'top_logprobs': []},
 {'token': ' computer',
  'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
  'logprob': -0.05092288,
  'top_logprobs': []},
 {'token': ' program',
  'bytes': [32, 112, 114, 111, 103, 114, 97, 109],
  'logprob': -4.7875077e-05,
  'top_logprobs': []},
 {'token': ',', 'bytes': [44], 'logprob': -0.07741755, 'top_logprobs': []},
 {'token': ' so',
  'bytes': [32, 115, 111],
  'logprob': -0.0013470192,
  'top_logprobs': []},
 {'token': ' I',
  'bytes': [32, 73],
  'logprob': -3.5716304e-05,
  'top_logprobs': []},
 {'token': ' don',
  'bytes': [32, 100, 111, 110],
  'logprob': -0.0013567866,
  'top_logprobs': []},
 {'token': "'t",
  'byt

In [9]:
msg.content

"I'm just a computer program, so I don't have feelings or emotions. How can I assist you today?"

<h3>LogProbs Also provides Token Counts</h3> 

In [10]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [11]:
msg=llmOpenAI.invoke(input=("human","how are you today?"))

In [12]:
msg.response_metadata['token_usage']

{'completion_tokens': 40,
 'prompt_tokens': 17,
 'total_tokens': 57,
 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}

<h3>Tracking Token Level Information</h3>

In [13]:
with get_openai_callback() as cb:
    result=llmGemini.invoke(input="Tell me a joke")
    print(cb)

Tokens Used: 21
	Prompt Tokens: 5
	Completion Tokens: 16
Successful Requests: 1
Total Cost (USD): $0.0


In [14]:
with get_openai_callback() as cb:
    result1=llmGemini.invoke(input="Tell me a joke")
    result2=llmOpenAI.invoke(input="Tell me a joke")
    print(cb)

Tokens Used: 48
	Prompt Tokens: 16
	Completion Tokens: 32
Successful Requests: 2
Total Cost (USD): $3.1e-05


In [49]:
with get_openai_callback() as cb:
    result1=llmGemini.invoke(input="Tell me a joke")
    result2=llmOpenAI.invoke(input="Tell me a joke")
    print(cb.total_tokens)

48


In [15]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a helpful assistant"),
        ("human", "{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)
tools=load_tools(tool_names=['wikipedia'])

In [16]:
agent=create_tool_calling_agent(llm=llmGemini,tools=tools,prompt=prompt)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [17]:
agentExecutor=AgentExecutor(agent=agent,verbose=True,stream_runnable=False,tools=tools)

In [18]:
with get_openai_callback() as cb:
    response=agentExecutor.invoke(input=
                                  {"input":"What's a hummingbird's scientific name and what's the fastest bird species?"}
                                 )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost: {cb.total_cost}")



> Entering new AgentExecutor chain...
{'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the WikipediaQuery tool.', 'properties': {'query': {'type_': 1, 'description': 'query to look up on wikipedia', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}

Invoking: `wikipedia` with `{'query': 'Hummingbird'}`


Page: Hummingbird
Summary: Hummingbirds are birds native to the Americas and comprise the biological family Trochilidae. With approximately 366 species and 113 genera, they occur from Alaska to Tierra del Fuego, but most species are found in Central and South America. As of 2024, 21

In [19]:
tools=load_tools(tool_names=["serpapi","llm-math"],llm=llmGemini)  # with wikipedia, it does not give the details
agent=create_tool_calling_agent(llm=llmGemini,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,verbose=True,stream_runnable=False,tools=tools)

In [20]:
with get_openai_callback() as cb:
    response=agentExecutor.invoke(input=
                                  {"input":"What's a hummingbird's scientific name and what's the fastest bird species?"}
                                 )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost: {cb.total_cost}")



> Entering new AgentExecutor chain...
{'name': 'Search', 'description': 'A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'properties': {'__arg1': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['__arg1'], 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'Calculator', 'description': 'Useful for when you need to answer questions about math.', 'parameters': {'type_': 6, 'properties': {'__arg1': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['__arg1'], 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.



Invoking: `Search` with `Hummingbird scientific name`


Hummingbirds belong to the order Apodiformes, meaning, "unfooted birds." There are three families in this order: Trochilidae (hummingbirds), Hemiprocnidae (tree swifts), and Apodidae (swifts). The family Trochilidae belongs to its own Suborder, Trochili.{'name': 'Search', 'description': 'A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'properties': {'__arg1': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['__arg1'], 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'Calculator', 'description': 'Useful for when you need to answer questions about math.', 'parameters': {'type_': 6, 'properties': {'__arg1': {'type_': 1, 'format_': '', 'description': '', 'nullable':

<h3>Using an LLM for the same activity</h3>

In [21]:
tools=load_tools(tool_names=["serpapi","llm-math"],llm=llmGemini)

In [22]:
agent=initialize_agent(tools=tools, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True,llm=llmGemini)

C:\Users\MSI 1\AppData\Local\Temp\ipykernel_8380\1306811143.py:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent=initialize_agent(tools=tools, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True,llm=llmGemini)


In [23]:
agentExecutor=AgentExecutor(agent=agent,verbose=True,stream_runnable=False,tools=tools)

In [24]:
with get_openai_callback() as cb:
    response=agent.invoke(
        input="Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost: {cb.total_cost}")



> Entering new AgentExecutor chain...
Thought: I need to find out who Olivia Wilde's current boyfriend is and then calculate his age raised to the 0.23 power. 
Action: Search
Action Input: Olivia Wilde boyfriend
Observation: In January 2021, Wilde began dating singer Harry Styles after they met during the filming of Don't Worry Darling.
Thought:Thought: I need to find out Harry Styles' current age.
Action: Search
Action Input: Harry Styles age
Observation: {'type': 'organic_result', 'title': 'Harry Styles › Age ShareSend feedbackClose menu'}
Thought:Thought: I need to find out Harry Styles' current age.
Action: Search
Action Input: Harry Styles age
Observation: {'type': 'organic_result', 'title': 'Harry Styles › Age ShareSend feedbackClose menu'}
Thought:Thought: I need to find out Harry Styles' current age.
Action: Search
Action Input: Harry Styles age
Observation: {'type': 'organic_result', 'title': 'Harry Styles › Age ShareSend feedbackClose menu'}
Thought:Thought: I need to find 